# 学习PyTorch-Day4

在训练神经网络时，最常用的算法是反向传播。在该算法中，根据损失函数相对于给定参数的梯度来调整参数（模型权重）

为了计算这些梯度，PyTorch 有一个内置的微分引擎，称为 __torch.autograd__。它支持任何计算图的梯度自动计算

- 例

In [9]:
import torch

device = torch.device(
    "mps"
    if torch.backends.mps.is_available()
    else
    "cuda"
    if torch.cuda.is_available()
    else
    "cpu"
)

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b

loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(loss)

tensor(1.1214, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [10]:
print("Gradient function for z = {}".format(z.grad_fn))
print("Gradient function for loss = {}".format(loss.grad_fn))

Gradient function for z = <AddBackward0 object at 0x7fd530a5b290>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fd530a5bd50>


- 计算梯度

In [11]:
loss.backward()

print(w.grad)
print(b.grad)

tensor([[0.1288, 0.2063, 0.2840],
        [0.1288, 0.2063, 0.2840],
        [0.1288, 0.2063, 0.2840],
        [0.1288, 0.2063, 0.2840],
        [0.1288, 0.2063, 0.2840]])
tensor([0.1288, 0.2063, 0.2840])


- __torch.no_grad()__

默认情况下，所有 require_grad=True 的张量都会跟踪其计算历史并支持梯度计算。 然而，在某些情况下，我们不需要这样做，例如，当我们训练了模型并且只想将其应用于某些输入数据时，即我们只想通过网络进行前向计算。 我们可以通过用 torch.no_grad() 块包围我们的计算代码来停止跟踪计算：

In [12]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


- __torch.detach()__

__torch.detach()__ 和 __torch.no_grad()__ 具有同样的功能

In [13]:
z = torch.matmul(x, w) + b
z_det = torch.detach(z)
print(z_det.requires_grad)

False


- 禁用梯度跟踪的情况
1. 将神经网络中的某些参数标记为冻结参数
2. 仅进行前向传递时加快计算速度，因为对不跟踪梯度的张量进行计算会更有效